In [1]:
##### -*-coding:utf-8 -*-
# import all the libraries 
# python==3.8; jupyterlab==3.0.12; lumicks.pylake==0.8.1; matplotlib==3.3.4; more-itertools==8.7.0;
# npTDMS==1.1.0; numpy==1.20.1; opencv-python==4.5.1.48; pandas==1.2.3; scipy==1.6.1; tifffile==2021.3.5
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from __future__ import division
import ruptures as rpt
import pwlf
from scipy import interpolate
from __future__ import division
from more_itertools import chunked
%matplotlib widget

In [2]:
# read and plot the binarized intensity trace calculated from image data
intens_name = input('please type in the file name:' ) + '.xlsx'
intens_path = input('please type in the file path:')
intens_file = os.path.join(intens_path, intens_name) 
intens = pd.read_excel(intens_file,sheet_name = 'Sheet1')
print(intens.head())

please type in the file name: image data-cycle#1-TimeIntensity
please type in the file path: C:\Users\86183\Desktop\20190529-012


   Unnamed: 0     time/s  intensity_binarized
0           0  11.086612                    1
1           1  11.299816                    1
2           2  11.513020                    1
3           3  11.726224                    1
4           4  11.939429                    1


In [3]:
intens_cycle = str('#') + input('please type in the processing cycle(e.g: 1):' )

please type in the processing cycle(e.g: 1): 1


In [4]:
# the following code blocks are intended to find out the bound protein events, 
# by splitting binarized intensity-time plots into sub-segments based on zero value
# step 1: reading the binarized intensity and time, followed by finding all the zero value points
time_intens = intens['time/s']
time_intens = np.array(time_intens)
intensity = intens['intensity_binarized']
intensity = np.array(intensity)
index = np.where(intensity==0)[0]
index = np.insert(index,0,0)
index = np.append(index,len(intensity))
# print(index)
# print(len(index))


In [6]:
# step 2: creating sub-segments from one zero-value to next zero-value
time_list = []
for i, value in enumerate(index):
    try:
        time_lis = time_intens[index[i]:index[i+1]]
    except IndexError:
        pass
    time_list.append(time_lis)   
# print(time_list)

In [7]:
# step 3: filtering the sub-segments without or only 1 one-value, 
# which indicates no protein occurs or protein occurs for only one pixel time
# this returns the bound events!!
filter_dots = 3 # default value is 3
time_lst = [x for x in time_list if x.size > filter_dots]
# print(len(time_lst))

In [8]:
# step 4: calculating the starting time and ending time of the segments
start_time = []
end_time = []
for arr in time_lst:
    start_time.append(arr[1])
    end_time.append(arr[-1])
print(start_time)
print(end_time)

[11.29981632653061, 29.20895918367347, 33.25983673469388, 47.54451020408163, 59.27073469387755, 83.14959183673469, 103.6171836734694, 110.8661224489796]
[27.29012244897959, 31.341, 39.22955102040816, 53.51422448979591, 77.17987755102041, 98.50028571428571, 108.7340816326531, 119.6074897959184]


In [9]:
# step 5: distinguish the exo and pol events based on force_transition_time
force_transi_time = input('please type in the force transision time in s:' ) 
force_transi_time = float(force_transi_time)
# to find out whether protein presents during force transition
from scipy import interpolate
func_1 = interpolate.interp1d(time_intens,intensity,kind='slinear',fill_value="extrapolate")
force_transi_intensity = int(func_1(force_transi_time))
# print(force_transi_intensity)

start_time_exo = []
end_time_exo = []
start_time_pol = []
end_time_pol = []
if force_transi_intensity == 0: 
    for i,value in enumerate(end_time):
        if value < force_transi_time:
            start_time_exo.append(start_time[i])
            end_time_exo.append(value)
    for i,value in enumerate(start_time):
        if value > force_transi_time:
            start_time_pol.append(value)             
            end_time_pol.append(end_time[i]) 
    start_time_pol = start_time_pol[:-1]
    end_time_pol = end_time_pol[:-1]
        
if force_transi_intensity == 1:
    for i,value in enumerate(end_time):
        if value < force_transi_time:
            start_time_exo.append(start_time[i])
            end_time_exo.append(value)
    start_time_exo.append(start_time[len(start_time_exo)])
    end_time_exo.append(force_transi_time)
    for i,value in enumerate(start_time):
        if value > force_transi_time:
            start_time_pol.append(value)
            end_time_pol.append(end_time[i]) 
    try:
        end_time_pol.insert(0,end_time[len(end_time_exo)-1])  
    except IndexError:
        pass
    if len(end_time_pol) == 0:
        end_time_pol.insert(0,end_time[-1])
    start_time_pol.insert(0,force_transi_time)
    
print(start_time_exo)
print(end_time_exo)
print(start_time_pol)
print(end_time_pol)

please type in the force transision time in s: 62.5


[11.29981632653061, 29.20895918367347, 33.25983673469388, 47.54451020408163, 59.27073469387755]
[27.29012244897959, 31.341, 39.22955102040816, 53.51422448979591, 62.5]
[62.5, 83.14959183673469, 103.6171836734694, 110.8661224489796]
[77.17987755102041, 98.50028571428571, 108.7340816326531, 119.6074897959184]


In [10]:
# step 6: distinguish the bound events and unbound events in exo and pol, respectively
# to calculate the dark events in exo region
dark_exo_start = []
dark_exo_end = []
if intensity[0] == 1 and force_transi_intensity == 0: #start with bound protein but lost during force transition 
    for i,value in enumerate(end_time_exo):
        try:
            dark_exo_start.append(value)
            dark_exo_end.append(start_time_exo[i+1])        
        except IndexError:
            pass
    dark_exo_end.append(force_transi_time)
    
if intensity[0] == 1 and force_transi_intensity == 1: #start with bound protein and have protein bound during force transition 
    for i,value in enumerate(end_time_exo):
        try:
            dark_exo_start.append(value)
            dark_exo_end.append(start_time_exo[i+1])        
        except IndexError:
            pass
    dark_exo_start = dark_exo_start[:-1]
    dark_exo_end = dark_exo_end[:]
    
if intensity[0] == 0 and force_transi_intensity == 0: #start without protein and have no protein bound during force transition 
    for i,value in enumerate(start_time_exo):
        try:        
            dark_exo_end.append(value)
            dark_exo_start.append(end_time_exo[i])
        except IndexError:
            pass
    dark_exo_start.insert(0,start_time_exo[0])
    dark_exo_end.append(force_transi_time)
    
if intensity[0] == 0 and force_transi_intensity == 1: #start without protein and have protein bound during force transition 
    for i,value in enumerate(start_time_exo):
        try:        
            dark_exo_end.append(value)
            dark_exo_start.append(end_time_exo[i])
        except IndexError:
            pass
    dark_exo_start.insert(0,start_time_exo[0])
    dark_exo_start = dark_exo_start[:-1]
    
print (dark_exo_start)
print (dark_exo_end)

[27.29012244897959, 31.341, 39.22955102040816, 53.51422448979591]
[29.20895918367347, 33.25983673469388, 47.54451020408163, 59.27073469387755]


In [11]:
# step 6: distinguish the bound events and unbound events in exo and pol, respectively
# to calculate the dark events in pol regin   
dark_pol_start = []
dark_pol_end = []
if intensity[-1] == 1 and force_transi_intensity == 0: #end with bound protein but have no protein bound during force transition 
    for i,value in enumerate(start_time_pol):
        try:
            dark_pol_end.append(value)
            dark_pol_start.append(end_time_pol[i])        
        except IndexError:
            pass
    dark_pol_start.insert(0,force_transi_time)
    dark_pol_start = dark_pol_start[:-1]
    
if intensity[-1] == 1 and force_transi_intensity == 1: #end with bound protein and have protein bound during force transition 
    for i,value in enumerate(end_time_pol):
        try:
            dark_pol_start.append(value)
            dark_pol_end.append(start_time_pol[i+1])        
        except IndexError:
            pass
    dark_pol_start = dark_pol_start[:-2]
    dark_pol_end = dark_pol_end[:-1]
    
if intensity[-1] == 0 and force_transi_intensity == 0: #end without protein and have no protein bound during force transition 
    for i,value in enumerate(start_time_pol):
        try:        
            dark_pol_end.append(value)
            dark_pol_start.append(end_time_pol[i])
        except IndexError:
            pass
    dark_pol_start.insert(0,force_transi_time)
    dark_pol_end.append(end_time_pol[-1])
    
if intensity[-1] == 0 and force_transi_intensity == 1: #end without protein but have protein bound during force transition 
    for i,value in enumerate(end_time_pol):
        try:
            dark_pol_start.append(value)
            dark_pol_end.append(start_time_pol[i+1])        
        except IndexError:
            pass
    dark_pol_end.append(end_time_pol[-1])
    
print (dark_pol_start)
print (dark_pol_end)

[77.17987755102041, 98.50028571428571, 108.7340816326531, 119.6074897959184]
[83.14959183673469, 103.6171836734694, 110.8661224489796, 119.6074897959184]


In [12]:
# Save all the analyzed data in an excel file
tem_path = intens_path + '\processing'
if not os.path.exists(tem_path):
    os.makedirs(tem_path)
    
excel_filename = tem_path + '\-' + '-cycle' + intens_cycle +"-SegmentedTrack" + '.xlsx'

writer = pd.ExcelWriter(excel_filename)

data_0 = {'track_Nmr':np.arange(start=1, stop=len(start_time), step=1),
        'start_time/s':start_time[:-1],
        'end_time/s':end_time[:-1],
       'track_duration/s':(np.array(end_time)-np.array(start_time))[:-1]}
data_1 = {'track_Nmr':np.arange(start=1, stop=len(start_time_exo)+1, step=1),
        'start_time/s':start_time_exo,
        'end_time/s':end_time_exo,
       'track_duration/s':(np.array(end_time_exo)-np.array(start_time_exo))}
data_2 = {'track_Nmr':np.arange(start=1, stop=len(start_time_pol)+1, step=1),
        'start_time/s':start_time_pol,
        'end_time/s':end_time_pol,
       'track_duration/s':(np.array(end_time_pol)-np.array(start_time_pol))}
data_3 = {'track_Nmr':np.arange(start=1, stop=len(dark_exo_start)+1, step=1),
        'dark_exo_start/s':dark_exo_start,
        'dark_exo_end/s':dark_exo_end,
       'track_duration/s':(np.array(dark_exo_end)-np.array(dark_exo_start))}
data_4 = {'track_Nmr':np.arange(start=1, stop=len(dark_pol_start)+1, step=1),
        'dark_pol_start/s':dark_pol_start[:],
        'dark_pol_end/s':dark_pol_end[:],
        'dark_track_duration/s':(np.array(dark_pol_end)-np.array(dark_pol_start))}

df_0 = pd.DataFrame(data_0)
df_1 = pd.DataFrame(data_1)
df_2 = pd.DataFrame(data_2)
df_3 = pd.DataFrame(data_3)
df_4 = pd.DataFrame(data_4)

df_0.to_excel(writer,sheet_name ='all_bound_tracks')
df_1.to_excel(writer,sheet_name ='bound_exo_track')
df_2.to_excel(writer,sheet_name ='bound_pol_track')
df_3.to_excel(writer,sheet_name ='dark_exo_track')
df_4.to_excel(writer,sheet_name ='dark_pol_track')
writer.save()

From this line on, code blocks below are intended for rupture detection

In [13]:
import os
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ruptures as rpt
from __future__ import division
from more_itertools import chunked
%matplotlib widget


In [14]:
# read and plot the basepair data
# name example: 20200304-160513 10pN- 30nM DNAp + 150nM_gp2.5 + 0.1mg_mL_BSA #019-002
name = input('please type in the file name:' ) + '.xlsx'
# path example: C:\Users\KTS260\Desktop
path = input('please type in the file path:')
filename = os.path.join(path, name) 

please type in the file name: force data-cycle#01-processedData
please type in the file path: C:\Users\86183\Desktop\20190529-012


In [15]:
data =  pd.read_excel(filename)
time = data['time']/1000
time = np.array(time)
basepairs = data['basepairs'] # 1D array
basepairs = np.array(basepairs)

fig, ax1 = plt.subplots(figsize=(10,4))
ax2 = ax1.twinx()
ax1.plot(time, basepairs, 'g-')
ax2.plot(time_intens,intensity,'b-')

ax1.set_xlabel('Time/s')
# ax1.set_ylim(3000,9000)
ax1.set_ylabel('Basepair/bp', color='g')
ax2.set_ylabel('Binarized Intensity', color='b')
ax2.set_ylim(-0.1,2)
plt.title('Example Basepairs+Binarized intensity')
#plt.plot(dbphat10_1_dt)
plt.show()
plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
# define a function PLRupture to calculate the speed, duration and processivity
# PLRupture is an improved changing point detection model based on rupture and piece-linear wise function
def PLRupture(basepairs,time_from,time_to,sigma):
    # define a temporary index to compute time of ROI, and subsuquent ROI of force, distance
    indtemp = np.where((time <= time_to) & (time >= time_from))
    time_ROI = time[indtemp]
    basepairs_ROI = basepairs[indtemp]

# From now on we work on change point detection
    signal = basepairs_ROI  # 1D array
# Apply proper model & searching method to work on change point detection
    model = "l2"  # "l1", "rbf", "linear", "normal", "ar",...

# For faster predictions, one can modify the jump parameter during initialization. The higher it is, the faster the prediction is achieved (at the expense of precision).
# There are plenty of models can be used for change point detection; but only rpt.Pelt seems to work with a algo.predict(pen=)
    algo = rpt.Pelt(model=model, jump=1).fit(signal)
# algo = rpt.BottomUp(model=model).fit(signal)
# algo = rpt.Window(width=40, model=model).fit(signal)
# algo = rpt.Binseg(model=model).fit(signal)
# algo = rpt.Dynp(model="rbf", params=params, jump=1, min_size=2).fit(signal)


# In our case, the number of change points is unknown, we need to specify a threshold on the residual norm using epsilon or a penalty using the pen parameter.
# n = number of samples
# sigma = noise standard deviation; the bigger the sigma, the less break points
# dim = dimention, in this case: dim = 1
    n = len(signal)

# my_bkps = algo.predict(epsilon=3 * n * sigma ** 2)
# # or
    dim = 1
# https://centre-borelli.github.io/ruptures-docs/user-guide/detection/bottomup/#:~:text=In%20the%20situation,n%20*%20sigma%20**%202)
# penelty constant = np.log(n) * dim * sigma ** 2
    my_bkps = algo.predict(pen=np.log(n) * dim * sigma ** 2)
## or you can assign a breakpoint number, e.g.:
# my_bkps = algo.predict(n_bkps=3)

# show results
#     fig, (ax,) = rpt.display(signal, my_bkps, figsize=(8, 4))
#     plt.show()
    # my_bkps returns index of break points
    
# trace back to time-basepair based on index
    time_bkp = np.zeros(len(my_bkps)-1)
    bps_bkp = np.zeros(len(my_bkps)-1)

# to exclude the last value
    time_bkp = time_ROI[my_bkps[0:-1]]
    bps_bkp = basepairs_ROI[my_bkps[0:-1]]

# to add the first value and last value to array
    time_bkp = np.insert(time_bkp,0,time_ROI[0])
    time_bkp = np.append(time_bkp,time_ROI[-1])
    bps_bkp = np.insert(bps_bkp,0,basepairs_ROI[0])
    bps_bkp = np.append(bps_bkp,basepairs_ROI[-1])
    
# initialize piecewise linear fit with your x and y data
    my_pwlf = pwlf.PiecewiseLinFit(time_ROI, basepairs_ROI)

# fit the data with the specified break points (ie the x locations of where
# the line segments should end
#     my_pwlf.fit_with_breaks(time_bkp)
# predict for the determined points
#     bps_bkp = my_pwlf.predict(time_bkp)

# Uses L-BFGS-B optimization to find the location of breakpoints from a guess of where breakpoint locations should be.
#     time_bkp = my_pwlf.fit_guess(time_bkp)

#     from scipy import interpolate
#     func_2 = interpolate.interp1d(time_ROI, basepairs_ROI,kind='slinear',fill_value="extrapolate")
#     bps_bkp = func_2(time_bkp)
    
# bks = np.append(time_bkp,bps_bkp, axis=-1).reshape((2,len(my_bkps)+1)).T
    duration_time = np.diff(time_bkp)
    processivity_event = np.diff(bps_bkp)
    speed = processivity_event / duration_time
    bound_duration = time_bkp[-1] - time_bkp[0]
    bound_processivity = bps_bkp[-1] - bps_bkp[0]
    return time_from,time_to,time_bkp,bps_bkp,duration_time,processivity_event,speed,bound_duration,bound_processivity

# The below code blocks needs to run 4 times, in order to calculate bound-exo,bound-pol and unbound-exo, unbound-pol, respectively

In [33]:
# analyze the exo or pol trace seperately
# Sigma value is determined with noise standard deviation; 
# in case not assigned, default sigma value in exo is 225, default sigma value in pol is 1500

# the below codes are used to specify with data fragment to be analyzed: e.g.:
# dark_pol_start, dark_exo_start, start_time_exo, start_time_pol can be used.
analyzing_frag = input('please type in the analyzing frag(e.g.:bound_exo, bound_pol,unbound_exo,unbound_pol):') 
if analyzing_frag == 'unbound_pol':
    analyzing_frag_start = dark_pol_start[1:]
    analyzing_frag_end = dark_pol_end[1:]
    sigma = 1500
if analyzing_frag == 'unbound_exo':
    analyzing_frag_start = dark_exo_start
    analyzing_frag_end = dark_exo_end
    sigma = 225
if analyzing_frag == 'bound_pol':
    analyzing_frag_start = start_time_pol
    analyzing_frag_end = end_time_pol
    sigma = 1500
if analyzing_frag == 'bound_exo':
    analyzing_frag_start = start_time_exo
    analyzing_frag_end = end_time_exo
    sigma = 225

# to double check if it is correct
print(analyzing_frag_start)
print(analyzing_frag_end)

please type in the analyzing frag(e.g.:bound_exo, bound_pol,unbound_exo,unbound_pol): unbound_pol


[98.50028571428571, 108.7340816326531, 119.6074897959184]
[103.6171836734694, 110.8661224489796, 119.6074897959184]


In [34]:
# the following code block is intended to calculate the ruptures, iterating all the sub-segments
time_bkp = []
bps_bkp = []
duration_time = []
processivity_event = []
speed = []
time_from = []
time_to = []
bound_duration = []
bound_processivity = []
for i,value in enumerate(analyzing_frag_start): #alternatively: dark_pol_start, dark_exo_start, start_time_exo, start_time_pol can be used.
    time_from_i = value
    time_to_i = analyzing_frag_end[i]  # alternatively: dark_pol_end, dark_exo_end, end_time_exo, end_time_exo would be used corresponded.
    time_from_i,time_to_i,time_bkp_i,bps_bkp_i,duration_time_i,processivity_event_i,speed_i,bound_duration_i,bound_processivity_i = PLRupture(basepairs,time_from_i,time_to_i,sigma)
    time_from = np.append(time_from,time_from_i)
    time_to = np.append(time_to,time_to_i)
    time_bkp = np.append(time_bkp,time_bkp_i)
    bps_bkp = np.append(bps_bkp,bps_bkp_i)
    duration_time = np.append(duration_time,duration_time_i)
    processivity_event = np.append(processivity_event,processivity_event_i)
    speed = np.append(speed,speed_i)
    bound_duration = np.append(bound_duration,bound_duration_i)
    bound_processivity = np.append(bound_processivity,bound_processivity_i)

# print(duration_time.size)
# print(processivity_event.size)
# print(speed.size)
# print(time_from.size)
# print(time_to.size)


<ipython-input-16-6e9ad1f7c245>:34: RuntimeWarning: divide by zero encountered in log
  my_bkps = algo.predict(pen=np.log(n) * dim * sigma ** 2)


ValueError: min() arg is an empty sequence

In [ ]:
# this is to plot the basepair changes and the detected changing-points as a function of time
fig, ax1 = plt.subplots(figsize=(10,4))
ax2 = ax1.twinx()
ax1.plot(time, basepairs, 'g-', label = 'basepairs')
ax1.scatter(time_bkp,bps_bkp,marker='*',c="r", label = "breaking points")
ax2.plot(time_intens,intensity,'lightgray', label = "Binarized intensity")

ax1.set_xlabel('Time/s')
# ax1.set_xlim(50,150)
ax1.set_ylabel('Basepair/bp', color='g')
ax2.set_ylabel('Binarized Intensity', color='lightgray')
ax2.set_ylim(-0.1,2)
ax1.legend()
ax2.legend(loc='center right')
plt.title('Example Basepairs+Binarized intensity')
plt.legend()
plt.show()
plt.tight_layout()

In [ ]:
# to save all the data    
excel_filename = intens_path + '\processing' + '\-' + '-cycle' + intens_cycle + name +'-'+ analyzing_frag + '.xlsx'
writer = pd.ExcelWriter(excel_filename)

data_1 = {'breaking time/s':time_bkp,
        'breaking position/bp':bps_bkp}
data_2 = {'event duration/s':duration_time,
        'event processivity/bp':processivity_event,
        'velocity/(bp/s)':speed}
data_3 = {'bound duration/s':bound_duration,
        'bound processivity/bp':bound_processivity}

df_1= pd.DataFrame(data_1)
df_2 = pd.DataFrame(data_2)
df_3 = pd.DataFrame(data_3)

df_1.to_excel(writer,sheet_name ='change_points', index=False)
df_2.to_excel(writer,sheet_name ='analyzed_data', index=False)
df_3.to_excel(writer,sheet_name ='processivity_one_burst', index=False)

writer.save()
plt.savefig(intens_path + '\processing' + '\-' + name +'-'+ analyzing_frag + '-Example Basepairs+Binarized intensity'+'.png', dpi=150)

In [ ]:
from scipy import stats
sem = stats.sem(speed, axis=None, ddof=0)
avg = np.mean(speed)

fig, ax = plt.subplots(figsize=(6, 4))
n_bins = 10
n, bins, patches = ax.hist(speed, n_bins)
ax.set_title('Rate')
ax.set_xlabel('Rate(bp/s)')
ax.set_ylabel('Occurrence')
plt.show()
plt.tight_layout()

print(avg)
print(sem)